<font size = 6>

<span><center>
    <i>Segmenting and Clustering Neighborhoods in Toronto</i></center></span>
    
</font>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 5>

   <a href="#item1">Assignment Part 1</a>

   <a href="#item2">Assignment Part 2</a>

   <a href="#item3">Assignment Part 3</a>
  
</font>
</div>

In [ ]:
#Installing required packages
'''pip install BeautifulSoup4
pip install lxml
pip install requests'''

In [2]:
from bs4 import BeautifulSoup # module for scrapping location data from web
import requests
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter('ignore')

# Assignment Part 1

In [3]:
#Importing postal codes and neighborhoods data of Canada from Wikipedia
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [4]:
#Parsing HTML data using BeautifulSoup module
soup = BeautifulSoup(source,'lxml')
soup.prettify()

'<!DOCTYPE html>\n<html class="client-nojs" dir="ltr" lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   List of postal codes of Canada: M - Wikipedia\n  </title>\n  <script>\n   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XpibQgpAAMMAAhUS-KMAAAAO","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":951325562,"wgRevisionId":951325562,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto"

Identified the ***table*** class that contained the required location data

In [5]:
table = soup.find('table', class_='wikitable')

### Converting HTML table data to dataframe

In [6]:
columns = []
for header in table.find_all('th'):
    columns.append(header.text)
columns

['Postal code\n', 'Borough\n', 'Neighborhood\n']

In [7]:
for i in range(0,len(columns)):
    columns[i] = columns[i].replace('\n','')
print(columns)

['Postal code', 'Borough', 'Neighborhood']


In [8]:
df = pd.DataFrame(columns = columns)

In [9]:
df.shape

(0, 3)

In [10]:
df.columns

Index(['Postal code', 'Borough', 'Neighborhood'], dtype='object')

Creating a dataframe from HTML table

In [11]:
rows = []
i=0
for sentence in table.find_all('tr'):  
    if(len(rows)!=0):
        rows = pd.Series(rows,index = df.columns)
        df = df.append(rows, ignore_index = True)
        rows = rows.tolist()
    rows.clear()
    for values in sentence.find_all('td'):
        rows.append(values.text.replace('\n',''))

rows = pd.Series(rows,index = df.columns)
df = df.append(rows, ignore_index = True)
print(df.shape, '\n', rows)

(180, 3) 
 Postal code              M9Z
Borough         Not assigned
Neighborhood                
dtype: object


In [12]:
df.describe()

,Postal code,Borough,Neighborhood
count,180,180,180
unique,180,11,99
top,M6H,Not assigned,
freq,1,77,77


Creating a dataframe with BOROUGHS that are Assigned

In [13]:
df_canada = pd.DataFrame(columns = columns)
df_canada = df[df.loc[:,'Borough'] != 'Not assigned'].copy()

In [14]:
df_canada.reset_index(inplace=True)
print(df_canada)

     index Postal code           Borough  \
0        2         M3A        North York   
1        3         M4A        North York   
2        4         M5A  Downtown Toronto   
3        5         M6A        North York   
4        6         M7A  Downtown Toronto   
..     ...         ...               ...   
98     160         M8X         Etobicoke   
99     165         M4Y  Downtown Toronto   
100    168         M7Y      East Toronto   
101    169         M8Y         Etobicoke   
102    178         M8Z         Etobicoke   

                                          Neighborhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                           Regent Park / Harbourfront  
3                    Lawrence Manor / Lawrence Heights  
4         Queen's Park / Ontario Provincial Government  
..                                                 ...  
98    The Kingsway / Montgomery Road  / Old Mill North  
99             

In [15]:
df_canada.drop(columns=['index'], inplace=True)

Replacing the `' /'` character in *Neighborhood* column with `','`

In [16]:
df_canada.loc[0:,'Neighborhood'] = df_canada.loc[0:,'Neighborhood'].apply(lambda value:value.replace(' /', ','))

### Dataframe with requirements as per question 1 of assignment

In [17]:
df_canada.shape

(103, 3)

In [18]:
df_canada.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Assignment Part 2

In [19]:
df_postalcodes = pd.read_csv('https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')

In [20]:
df_postalcodes

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [21]:
for i, code in enumerate(df_canada.loc[0:, 'Postal code']):
    for j, code2 in enumerate(df_postalcodes.loc[0:, 'Postal Code']):
        if code == code2:
            df_canada.loc[i,'Latitude'] = df_postalcodes.loc[j,'Latitude']
            df_canada.loc[i,'Longitude'] = df_postalcodes.loc[j,'Longitude']
print(df_canada.head(2))

  Postal code     Borough      Neighborhood   Latitude  Longitude
0         M3A  North York         Parkwoods  43.753259 -79.329656
1         M4A  North York  Victoria Village  43.725882 -79.315572


### Final data frame as per requirement for  question 2 of Assignment

In [22]:
df_canada

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Assignment Part 3

Installing required packages for clustering neighborhoods in Toronto

In [23]:
#Plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#module for clustering
from sklearn.cluster import KMeans

#Mapping
import folium

#Module for getting location data for address values
from geopy.geocoders import Nominatim

In [24]:
print(' The dataframe has {} Boroughs and {} neighborhoods'. format(len(df_canada['Borough'].unique()), df_canada.shape[0]))

 The dataframe has 10 Boroughs and 103 neighborhoods


In [25]:
df_canada['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [26]:
df_toronto = df_canada[df_canada.loc[0:,'Borough'].str.contains('Toronto', case=False)]

In [27]:
df_toronto.reset_index(inplace = True)

In [28]:
df_toronto.drop(columns = ['index'],inplace = True)

In [29]:
df_toronto.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [30]:
print('There are {} boroughs and {} neighborhoods in Toronto'.format(len(df_toronto.Borough.unique()),df_toronto.shape[0]))

There are 4 boroughs and 39 neighborhoods in Toronto


In [31]:
address = 'Toronto'

geolocator = Nominatim(user_agent = 'Toronto locator')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The latitude and longitude of Toronto city are {}, {}'.format(latitude, longitude))

The latitude and longitude of Toronto city are 43.6534817, -79.3839347


Mapping Toronto's neighborhoods using *Folium*

In [32]:
map_toronto = folium.Map(location=[latitude,longitude], zoom_starts=20)

for lat, long, Borough, Neighborhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],
                                            df_toronto['Borough'],df_toronto['Neighborhood']):
    label = '{},{}'.format(Borough,Neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,long],
                       radius=5,
                       popup=label,
                       color='blue',
                       fill=True,
                       fill_color='yellow',
                       fill_opacity=0.8,
                       parse_html=True).add_to(map_toronto)
map_toronto    

In [33]:
CLIENT_ID = '<CLIENT_ID>' # my Foursquare ID
CLIENT_SECRET = '<CLIENT SECRET>' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: <CLIENT_ID>
CLIENT_SECRET:<CLIENT SECRET>


In [34]:
def getNearbyVenues(access_token, LIMIT, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?oauth_token={}&v={}&ll={},{}&radius={}&limit={}'.format(
            access_token, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
token = 'RBPLNFEVFSX5N12KWCQY35ZZNPIDEE0Q5WWWSEFMJLICKMD3'
limit = 100
toronto_venues = getNearbyVenues(token, limit,df_toronto['Neighborhood'],df_toronto['Latitude'],df_toronto['Longitude'])
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
...,...,...,...,...,...,...,...
1812,Business reply mail Processing CentrE,43.662744,-79.321558,Greenwood Cigar & Variety,43.664538,-79.325379,Smoke Shop
1813,Business reply mail Processing CentrE,43.662744,-79.321558,Central Toronto Wrestling Club,43.664398,-79.325937,Martial Arts Dojo
1814,Business reply mail Processing CentrE,43.662744,-79.321558,Revolution Recording,43.662561,-79.326940,Recording Studio
1815,Business reply mail Processing CentrE,43.662744,-79.321558,ONE Academy,43.662253,-79.326911,Gym / Fitness Center


In [36]:
toronto_venues.shape

(1817, 7)

List of number of venues per neighborhood

In [37]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,67,67,67,67,67,67
"Brockton, Parkdale Village, Exhibition Place",15,15,15,15,15,15
Business reply mail Processing CentrE,23,23,23,23,23,23
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",19,19,19,19,19,19
Central Bay Street,86,86,86,86,86,86
Christie,30,30,30,30,30,30
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,54,54,54,54,54,54


In [38]:
print('There are {} venue categories in and around Toronto'.format(len(toronto_venues['Venue Category'].unique())))

There are 254 venue categories in and around Toronto


Creating dataframe based on the *type of venues* present in each neigborhood in Toronto

In [39]:
toronto_onehot=pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

In [40]:
toronto_onehot.head()

,ATM,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
toronto_onehot['Neighborhood']=toronto_venues['Neighborhood']

In [42]:
list_n = list(toronto_onehot.columns)

In [43]:
for i, value in enumerate(list_n):
    if value == 'Neighborhood':
        print(i)

174


In [44]:
fixed_columns = [toronto_onehot.columns[174]]+list(toronto_onehot.columns[:174])+list(toronto_onehot.columns[175:])
toronto_onehot = toronto_onehot[fixed_columns]

In [45]:
toronto_onehot.head(3)

,Neighborhood,ATM,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
toronto_onehot.shape

(1817, 254)

In [47]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,ATM,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.014925,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000
2,Business reply mail Processing CentrE,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.043478
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.00,0.00,0.052632,0.052632,0.052632,0.105263,0.157895,0.105263,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000
4,Central Bay Street,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.011628,0.00,0.000000,0.011628,0.00,0.000000,0.00,0.011628
5,Christie,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000
6,Church and Wellesley,0.000000,0.01,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.01,0.010000,0.00,0.030000
7,"Commerce Court, Victoria Hotel",0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.020000,0.00,0.000000,0.010000,0.00,0.000000,0.00,0.000000
8,Davisville,0.018519,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.018519,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000
9,Davisville North,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000


In [48]:
toronto_grouped.shape

(39, 254)

#### Listing popular venues per Neighborhood

In [49]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.04
2    Restaurant  0.03
3      Beer Bar  0.03
4          Café  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0  Breakfast Spot  0.13
1     Coffee Shop  0.13
2    Climbing Gym  0.07
3         Stadium  0.07
4            Café  0.07


----Business reply mail Processing CentrE----
                  venue  freq
0  Fast Food Restaurant  0.09
1    Light Rail Station  0.09
2  Gym / Fitness Center  0.09
3           Yoga Studio  0.04
4                Garden  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.16
1     Airport Terminal  0.11
2  Rental Car Location  0.11
3       Airport Lounge  0.11
4        Boat or Ferry  0.05


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.14
1                Café  0.06
2      Ice Cream 

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Creating dataframe with most common venues for each neighborhood

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sortd = pd.DataFrame(columns=columns)
neighborhoods_venues_sortd['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sortd.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sortd.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Italian Restaurant,French Restaurant,Beer Bar,Farmers Market,Seafood Restaurant,Restaurant,Bakery,Café
1,"Brockton, Parkdale Village, Exhibition Place",Breakfast Spot,Coffee Shop,Performing Arts Venue,Bakery,Italian Restaurant,Café,Bar,Furniture / Home Store,Stadium,Restaurant
2,Business reply mail Processing CentrE,Fast Food Restaurant,Gym / Fitness Center,Light Rail Station,Brewery,Farmers Market,Spa,Smoke Shop,Skate Park,Martial Arts Dojo,Yoga Studio
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Rental Car Location,Airport Lounge,Boutique,Plane,Sculpture Garden,Coffee Shop,Harbor / Marina,Airport Gate
4,Central Bay Street,Coffee Shop,Café,Sushi Restaurant,Ice Cream Shop,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Thai Restaurant,Bakery,Chinese Restaurant


##### Clustering based on the new dataframe with venues

In [52]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [53]:
# add clustering labels
neighborhoods_venues_sortd.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sortd.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Café,Bakery,Park,Theater,Mexican Restaurant,Breakfast Spot,Performing Arts Venue,Italian Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Diner,Japanese Restaurant,Burger Joint,Distribution Center,Spa,Fast Food Restaurant,Beer Bar,Seafood Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Middle Eastern Restaurant,Italian Restaurant,Cosmetics Shop,Japanese Restaurant,Burger Joint,Fast Food Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Restaurant,Italian Restaurant,Hotel,Breakfast Spot,Cosmetics Shop,Gastropub,Bakery,Diner
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pizza Place,Park,Pub,Home Service,Trail,Health Food Store,Coffee Shop,Asian Restaurant,Electronics Store,Eastern European Restaurant


In [54]:
toronto_merged.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Café,Bakery,Park,Theater,Mexican Restaurant,Breakfast Spot,Performing Arts Venue,Italian Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Diner,Japanese Restaurant,Burger Joint,Distribution Center,Spa,Fast Food Restaurant,Beer Bar,Seafood Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Middle Eastern Restaurant,Italian Restaurant,Cosmetics Shop,Japanese Restaurant,Burger Joint,Fast Food Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Restaurant,Italian Restaurant,Hotel,Breakfast Spot,Cosmetics Shop,Gastropub,Bakery,Diner
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pizza Place,Park,Pub,Home Service,Trail,Health Food Store,Coffee Shop,Asian Restaurant,Electronics Store,Eastern European Restaurant


#### Mapping identified clusters using *Folium* **markers**

In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,0,Pizza Place,Park,Pub,Home Service,Trail,Health Food Store,Coffee Shop,Asian Restaurant,Electronics Store,Eastern European Restaurant
19,Central Toronto,0,Home Service,Health & Beauty Service,Ice Cream Shop,Pool,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
21,Central Toronto,0,Bus Line,Park,Sushi Restaurant,Jewelry Store,Home Service,Trail,Donut Shop,Discount Store,Distribution Center,Dog Run


### Cluster 2

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Pub,Café,Bakery,Park,Theater,Mexican Restaurant,Breakfast Spot,Performing Arts Venue,Italian Restaurant
1,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Diner,Japanese Restaurant,Burger Joint,Distribution Center,Spa,Fast Food Restaurant,Beer Bar,Seafood Restaurant
2,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Middle Eastern Restaurant,Italian Restaurant,Cosmetics Shop,Japanese Restaurant,Burger Joint,Fast Food Restaurant
3,Downtown Toronto,1,Café,Coffee Shop,Restaurant,Italian Restaurant,Hotel,Breakfast Spot,Cosmetics Shop,Gastropub,Bakery,Diner
5,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Italian Restaurant,French Restaurant,Beer Bar,Farmers Market,Seafood Restaurant,Restaurant,Bakery,Café
6,Downtown Toronto,1,Coffee Shop,Café,Sushi Restaurant,Ice Cream Shop,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Thai Restaurant,Bakery,Chinese Restaurant
7,Downtown Toronto,1,Grocery Store,Café,Rental Car Location,Coffee Shop,Park,Italian Restaurant,Nightclub,Diner,Bank,Japanese Restaurant
8,Downtown Toronto,1,Coffee Shop,Restaurant,Thai Restaurant,American Restaurant,Café,Sushi Restaurant,Gym,Asian Restaurant,Steakhouse,Pizza Place
9,West Toronto,1,Pharmacy,Bakery,Pizza Place,Bar,Bank,Supermarket,Bus Stop,Brazilian Restaurant,Athletics & Sports,Recording Studio
10,Downtown Toronto,1,Coffee Shop,Aquarium,Café,Hotel,Scenic Lookout,Restaurant,Italian Restaurant,Brewery,Bar,History Museum


### Cluster 3

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,2,Tennis Court,Playground,Park,Beer Store,Comfort Food Restaurant,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant
33,Downtown Toronto,2,Park,Trail,Playground,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


### Cluster 4

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Gym / Fitness Center,Park,Lawyer,Bus Line,Comic Shop,Concert Hall,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store


#### Obeservation based on the clusters formed
The clustering algorithm is influenced due to the presence of coffee shops since most of the neighborhoods are grouped under one cluster that majorly favored Coffee shops/Cafe`s